In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer

# Data base import

In [ ]:
from utils import sample_data

NB_SAMPLES = 10
PATH_SAMPLE = Path("../data/samples/sample_{}.csv".format(NB_SAMPLES))
PATH_CSV = Path("../data/train.csv")

SMALL_DATA = sample_data.SampleData(
    PATH_CSV, nb_samples=NB_SAMPLES,
    save=True, path_save=PATH_SAMPLE
)

DATAFRAME = pd.read_csv(PATH_SAMPLE)
DATA = pd.DataFrame.to_numpy(DATAFRAME)

In [ ]:
print(DATAFRAME)

# Pre-processing

In [ ]:
from utils.load_data import load_data

ALPHANUM_ONLY = False
WORD_SIZE = 30
SENTENCE_SIZE = 50
FILL_WITH = "$"
SPLIT_PUNCTUATION = False # to tell wheter the puncutation "!?.;,/" etc are keeped sticked to a word or not

X_STRING, X_SCALAR, Y = load_data(DATA, WORD_SIZE, SENTENCE_SIZE, FILL_WITH, SPLIT_PUNCTUATION, ALPHANUM_ONLY)

print(DATA.shape, X_STRING.shape, Y.shape, X_SCALAR.shape)

print("\nOriginal data :")
print(DATA[0])
print("\nFilled sentence :")
print(X_STRING[0])
print("\nLabel :")
print(Y[0])
print("\nDescriptor :")
print(X_SCALAR[0])

# Classification

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
nb_neighbors = 2

knn = KNeighborsRegressor(nb_neighbors, weights="distance")
knn.fit(X_SCALAR, Y)

In [ ]:
from utils.post_processing import pred_to_sentence, sentence_to_string, filter_character

pred = knn.predict(X_SCALAR)
meaning_sentences = pred_to_sentence(X_STRING, pred)

results = []
for sentence in meaning_sentences:
    result = ""
    for word in sentence:
        filtered_word = filter_character(word, "$")
        if len(filtered_word) != 0:
            result += filtered_word + " "
    results.append(result)
results=np.array(results, dtype=object)

In [ ]:
print(results)

In [ ]:
print(DATA[:, 2])

In [ ]:
from utils.loss import jaccard

avg = 0
for i in range(len(results)):
    print(avg)
    avg += jaccard(results[i], SMALL_DATA.sample_data["selected_text"].to_numpy()[i])
avg /= len(results)

print(avg)